In [67]:
import pandas as pd
import os

In [68]:
# NOTE: THIS PATH READS TO THE LOCATION OF THE LARGE FILE USB DRIVE ON A MAC,
# MAY NEED TO BE REDIRECTED TO THE APPROPRIATE LOCATION ON THE USER'S MACHINE
absolute_filepath = '/Volumes/Lexar/2010 data/'

In [69]:
# Context columns to drop from housing and median dfs
drop_cols_housing = ['REGIONA','DIVISIONA','STATEA','COUNTYA','COUSUBA','PLACEA','RES_ONLYA',
                     'TRUSTA','AIANHHA','AITSCEA','TTRACTA','TBLKGRPA','ANRCA','NECTAA',
                     'NECTADIVA','CNECTAA','UAA','URBRURALA','CDA','SLDUA','SLDLA','ZCTA5A',
                     'SUBMCDA','SDELMA','SDSECA','SDUNIA','SABINSA','METDIVA', 'CSAA', 'NAME',
                     'IFC001','IFG001','IFG002','IFG003','IFG004','IFG005','IFG006','IFG007',
                     'IFG008','IFH001','IFH002','IFH003','IFH004','IFH005','IFH006','IFH007',
                     'IFH008']

drop_cols_median = ['YEAR','REGIONA','DIVISIONA','STATE','STATEA','COUNTY','COUNTYA','COUSUBA',
                    'PLACEA','TRACTA','BLKGRPA','CONCITA','AIANHHA','RES_ONLYA','TRUSTA','AITSCEA',
                    'ANRCA','CBSAA','CSAA','METDIVA','NECTAA','CNECTAA','NECTADIVA','UAA',
                    'CDCURRA','SLDUA','SLDLA','SUBMCDA','SDELMA','SDSECA','SDUNIA','PUMA5A',
                    'BTTRA','BTBGA','NAME_M', 'NAME_E', 'JTIM001']

In [70]:
# Dictionary of column name mappings
housing_col_mapper = {'H7X001':'Total Pop','H7X002':'White alone','H7X003':'Black or African American alone',
                      'H7X004':'American Indian and Alaska Native alone', 'H7X005':'Asian alone',
                      'H7X006':'Native Hawaiian and Other Pacific Islander alone', 
                      'H7X007':'Some Other Race alone','H7X008':'Two or More Races', 
                      'IFE001':'Total Units','IFE002':'Occupied','IFE003':'Vacant',
                      'IFF001':'Total Occupied Units','IFF002':'Owned with a mortgage or a loan',
                      'IFF003':'Owned free and clear','IFF004':'Renter occupied', 
                      'BLKGRPA': 'BLCK_GRPA', 'CONCITA':'C_CITYA'}

median_col_mapper = {'JTIE001':'Median'}

In [71]:
# Chicago's CBSA code 
chicago_cbsa_code = 16980

In [72]:
# Load housing data into df
if os.path.exists(absolute_filepath + 'nhgis0012_ds172_2010_block.csv'):
    housing_df = pd.read_csv(absolute_filepath + 'nhgis0012_ds172_2010_block.csv', 
                             low_memory=False).drop(drop_cols_housing, axis = 1)
else:
    raise IOError('This file does not exists in this location')

In [73]:
# Load median data into df
if os.path.exists(absolute_filepath + 'nhgis0012_ds176_20105_2010_tract.csv'):
    median_df = pd.read_csv(absolute_filepath + 'nhgis0012_ds176_20105_2010_tract.csv', 
                            encoding = "ISO-8859-1").drop(drop_cols_median, axis = 1)
else:
    raise IOError('This file does not exists in this location')

In [74]:
# Subset on Chicago CBSA code
chi_housing_df = housing_df[housing_df['CBSAA'] == chicago_cbsa_code].copy()
chi_housing_df.rename(columns = housing_col_mapper, inplace = True)

In [75]:
# Rename columns in the median df
median_df.rename(columns = median_col_mapper, inplace = True)

In [76]:
# Shorten GISJOIN field to exclude block identifiers so that it can be used to match values
gis_short = chi_housing_df['GISJOIN'].str[:14]
chi_housing_df = chi_housing_df.assign(GIS_SHORT = gis_short)
chi_housing_df.rename(columns = {'GISJOIN': 'GISJOIN_BLOCK'}, inplace = True)

In [77]:
# Merge median and raw dfs on shortened GISJOIN field
full_df = chi_housing_df.merge(median_df,left_on = ['GIS_SHORT'], right_on = ['GISJOIN'])

In [78]:
# Combine race categories
full_df['Other races'] = full_df[['American Indian and Alaska Native alone', 'Asian alone',
       'Native Hawaiian and Other Pacific Islander alone',
       'Some Other Race alone', 'Two or More Races']].sum(axis = 1)
full_df = full_df.drop(['American Indian and Alaska Native alone', 'Asian alone',
       'Native Hawaiian and Other Pacific Islander alone',
       'Some Other Race alone', 'Two or More Races'], axis = 1)

In [79]:
# Calculate race percentages
full_df['Pct_White'] = full_df['White alone']/full_df['Total Pop']
full_df['Pct_Black'] = full_df['Black or African American alone']/full_df['Total Pop']
full_df['Pct_Other'] = full_df['Other races']/full_df['Total Pop']
full_df = full_df.drop(['White alone', 'Black or African American alone','Other races'], axis = 1)

In [80]:
# Calculate unit occupancy/vacancy percentages
full_df['Owner occupied'] = full_df[['Owned with a mortgage or a loan',
                                     'Owned free and clear']].sum(axis = 1)
full_df['% Occupied'] = full_df['Total Occupied Units']/full_df['Total Units']
full_df['% Vacant'] = full_df['Vacant']/full_df['Total Units']
full_df['% Owner Occupied'] = full_df['Owner occupied']/full_df['Total Occupied Units']
full_df['% Renter'] = full_df['Renter occupied']/full_df['Total Occupied Units']
full_df = full_df.drop(['Occupied', 'Vacant', 'Owned with a mortgage or a loan',
                        'Owned free and clear', 'Renter occupied'], axis = 1)

In [81]:
# Update index column name to match other years, set as index
full_df.drop(['GISJOIN', 'GIS_SHORT'], axis = 1, inplace = True)
full_df.rename(columns = {'GISJOIN_BLOCK': 'GISJOIN'}, inplace = True)
full_df.set_index(keys = 'GISJOIN', inplace = True)

In [82]:
# Reorder columns to match other years
reorder = ['YEAR', 'BLOCKA', 'BLCK_GRPA', 'TRACTA', 'C_CITYA', 'COUNTY',
       'CBSAA', 'STATE', 'Total Pop', 'Pct_White', 'Pct_Black',
       'Pct_Other', 'Total Units', 'Median', '% Occupied', '% Vacant',
       '% Owner Occupied', '% Renter']

full_df = full_df[reorder]

In [83]:
# Write df out to CSV
full_df.to_csv("../2010_census_data.csv", sep = "|")